# QCArchive+QCMLForge Demo with CyberShuttle

The first half of this demo shows how to use QCArchive to setup a dataset
and run computations with ease. The compute resource for this noteboook
uses Cybershuttle; however, a purely local resource demo is available under
`./demo_local.ipynb`.

The second half of this demo shows how one can consume the generated data
to train AP-Net models through QCMLForge. 

## How is this useful?
Prior to using quantum mechanical (QM) methods for applications, often computational
chemists will either consult previous studies of benchmarked methods on similar
systems, or perform the benchmarking task themselves. Then after quantifying
error, the level of theory that balances both expected error and is a reasonable
computational cost will be selected for applying to novel system(s).

The Sherrill research group has performed quite a few of these benchmarking
studies particularly for studying intermolecular interaction energies. These
interaction energies basically determine how attractive (or repulsive) molecules
behave when brought close together. A more practical definition for an
interaction energy (IE) is: 

$E_{\rm IE} = E_{\rm dimer} - E_{\rm monomerA} - E_{\rm monomerB}$.

Hence, the interaction energy is defined as the energy difference between
the dimer (when the two molecules are together) and the molecules on
their own (monomerA and monomerB in isolation).

To demonstrate how one might carryout the computational aspects of a
benchmarking study, the present notebook will provide examples of managing
datasets with QCArchive and running QM calculations prior to analyzing error
statistics with respect to reference energies. The second half of the notebook
provides examples on how you could then apply QM data towards training ML models
through QCMLForge.

In [1]:
# !pip install --force-reinstall -q "airavata-python-sdk[notebook]"
import airavata_jupyter_magic


Loaded airavata_jupyter_magic (2.0.17) 
(current runtime = local)

  %authenticate                              -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]               -- Request a runtime named <rt> with configuration <args>.
                                                Call multiple times to request multiple runtimes.
  %restart_runtime <rt>                      -- Restart runtime <rt> if it hangs. This will clear all variables.
  %stop_runtime <rt>                         -- Stop runtime <rt> when no longer needed.
  %wait_for_runtime <rt>                     -- Wait for runtime <rt> to be ready.
  %switch_runtime <rt>                       -- Switch the active runtime to <rt>. All subsequent cells will run here.
  %%run_on <rt>                              -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>                         -- Show the status of runtime <rt>.
  %copy_data source=<r1:f

In [2]:
%authenticate

Output()

Authenticated.

In [ ]:
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared
%wait_for_runtime hpc_cpu
%switch_runtime hpc_cpu

Requesting runtime=hpc_cpu...
[expanse:shared, 60 Minutes, 1 Node(s), 24 CPU(s), 0 GPU(s), 48000 MB RAM, 0 MB VRAM]
* modules=[]
* libraries=['numpy=2.2.5', 'pandas=2.2.3', 'pip', 'psycopg2=2.9.9', 'postgresql=17.4', 'pytest', 'python=3.10', 'psi4=1.9.1', 'pytorch-cpu=2.5.1', 'jupyter=1.1.1', 'requests', 'setuptools', 'torchaudio=2.5.1', 'torchvision=0.20.1', 'pytorch_geometric=2.6.1', 'pytorch_scatter=2.1.2=cpu*', 'pytorch-minimize=0.0.2', 'matplotlib=3.10.1', 'pydantic=1', 'scipy=1.15.*', 'tqdm']
* pip=['cdsg-tools==0.0.2', 'qm-tools-aw==1.4.5', 'qcmlforge==0.0.6', 'qcfractal==0.59', 'qcmanybody', 'qcfractalcompute==0.59']
* mounts=[]


Requested runtime=hpc_cpu


Output()

Switched to runtime=hpc_cpu.


In [11]:
%copy_data source=local:setup_qcfractal.py target=hpc_cpu:setup_qcfractal.py
%copy_data source=local:__init__.py target=hpc_cpu:__init__.py
%copy_data source=local:combined_df_subset_358.pkl target=hpc_cpu:combined_df_subset_358.pkl
%ls

Copying from local:setup_qcfractal.py to hpc_cpu:setup_qcfractal.py
local:/home/awallace43/gits/cybershuttle_demo/setup_qcfractal.py --> hpc_cpu:setup_qcfractal.py... [200]
Copying from local:__init__.py to hpc_cpu:__init__.py
local:/home/awallace43/gits/cybershuttle_demo/__init__.py --> hpc_cpu:__init__.py... [200]
Copying from local:combined_df_subset_358.pkl to hpc_cpu:combined_df_subset_358.pkl
local:/home/awallace43/gits/cybershuttle_demo/combined_df_subset_358.pkl --> hpc_cpu:combined_df_subset_358.pkl... [200]
apnet2_transfer_learning.pt  demo_local.ipynb  __init__.py  s22.py
combined_df_subset_358.pkl   demo_og.py        LICENSE      setup_qcfractal.py
cybershuttle.yml             demo.py           main.py
demo.html                    environment.yml   notes.md
demo.ipynb                   example_plots/    README.md


In [ ]:
!ls -l
!cat __init__.py
!cat setup_qcfractal.py
!pwd

⠧ Connecting to=hpc_cpu... status=CONNECTED
total 19525
-rw-r--r-- 1 gridchem uic151       29 May  5 10:27 A447103629
-rw-r--r-- 1 gridchem uic151    79955 May  5 11:02 AiravataAgent.stderr
-rw-r--r-- 1 gridchem uic151    14014 May  5 11:02 AiravataAgent.stdout
drwxr-xr-x 2 gridchem uic151    57344 May  5 10:58 __init__.py
-rwxr-xr-x 1 gridchem uic151 17640299 May  2 02:31 airavata-agent
lrwxrwxrwx 1 gridchem uic151       39 May  5 10:27 application -> /home/gridchem/cybershuttle/application
drwxr-xr-x 2 gridchem uic151    57344 May  5 10:59 combined_df_subset_358.pkl
-rw-r--r-- 1 gridchem uic151     1851 May  5 10:27 job_685473831.slurm
-rw-r--r-- 1 gridchem uic151     4137 May  2 02:32 kernel.py
-rwxr-xr-x 1 gridchem uic151 16371024 Apr  9 05:04 micromamba
drwxr-xr-x 2 gridchem uic151    57344 May  5 10:58 setup_qcfractal.py
cat: __init__.py: Is a directory
cat: setup_qcfractal.py: Is a directory
/home/gridchem/cybershuttle/workspace/uic151/PROCESS_173a7907-87bc-4bd7-9123-844fe949b5e

In [6]:
import psi4
from pprint import pprint as pp
import pandas as pd
import numpy as np
from qm_tools_aw import tools
import matplotlib.pyplot as plt
# QCElemental Imports
from qcelemental.models import Molecule
import qcelemental as qcel
# Dataset Imports
from qcportal import PortalClient
from qcportal.singlepoint import SinglepointDatasetEntry, QCSpecification
from qcportal.manybody import ManybodyDatasetEntry, ManybodySpecification
from torch import manual_seed

manual_seed(42)

h2kcalmol = qcel.constants.hartree2kcalmol
print('Imports')

⠇ Connecting to=hpc_cpu... status=CONNECTED
Imports


# QCArchive Setup
QCArchive uses a PostgreSQL database for storing QM data including geometries,
energies, and properties. Furthermore, to save on compute resources, every job
generates a unique hash allowing future computations to be able to query previous
job results for avoiding re-computing calculations, unless specifically requested.

The following function initializes the QCArchive DB and QCFractal server, allowing
us to then start the services for interfacing them through python. The configurations
below will operate on a remote node through cybershuttle.

In [19]:
from .setup_qcfractal import setup_qcarchive_qcfractal
import os

# Update these if you request non-default resources from cybershuttle.yml
max_workers = 3
cores_per_worker = 8
memory_per_worker = 16

setup_qcarchive_qcfractal(
    QCF_BASE_FOLDER=os.path.join(os.getcwd(), "qcfractal"),
    start=False,
    reset=False,
    db_config={
        "name": None,
        "enable_security": "false",
        "allow_unauthenticated_read": None,
        "logfile": None,
        "loglevel": None,
        "service_frequency": 5,
        "max_active_services": None,
        "heartbeat_frequency": 60,
        "log_access": None,
        "database": {
            "base_folder": None,
            "host": None,
            "port": 5433,
            "database_name": "qca",
            "username": None,
            "password": None,
            "own": None,
        },
        "api": {
            "host": None,
            "port": 7778,
            "secret_key": None,
            "jwt_secret_key": None,
        },
    },
    resources_config={
            "update_frequency": 5,
            "cores_per_worker": cores_per_worker,
            "max_workers": max_workers,
            "memory_per_worker": memory_per_worker,
    },
    conda_env=None,
    worker_sh=None,
)

⠇ Connecting to=hpc_cpu... status=CONNECTED


In [ ]:
get_ipython().system = os.system
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start > qcfractal/qcf_server.log &

In [ ]:
!qcfractal-compute-manager --config=`pwd`/qcfractal/resources.yml &

⠹ Connecting to=hpc_cpu.....


InvalidStateError: Runtime=hpc_cpu is in state=EXPERIMENT_COMPLETED
process  completed

STDOUT:
CS_HOME=/home/gridchem/cybershuttle
AGENT=agent_4b90d99e-04a5-4707-ac60-72cb2f64034e
SERVER=api.gateway.cybershuttle.org
CONTAINER=remote-agent-base.sif
LIBRARIES=numpy=2.2.5,pandas=2.2.3,pip,psycopg2=2.9.9,postgresql=17.4,pytest,python=3.10,psi4=1.9.1,pytorch-cpu=2.5.1,jupyter=1.1.1,requests,setuptools,torchaudio=2.5.1,torchvision=0.20.1,pytorch_geometric=2.6.1,pytorch_scatter=2.1.2=cpu*,pytorch-minimize=0.0.2,matplotlib=3.10.1,pydantic=1,scipy=1.15.*,tqdm
PIP=cdsg-tools==0.0.2,qm-tools-aw==1.4.5,qcmlforge==0.0.6,qcfractal==0.59,qcmanybody,qcfractalcompute==0.59
MOUNTS=


Transaction

  Prefix: /dev/shm/cybershuttle/envs/18dccede

  Updating specs:

   - numpy=2.2.5
   - pandas=2.2.3
   - pip
   - psycopg2=2.9.9
   - postgresql=17.4
   - pytest
   - python=3.10
   - psi4=1.9.1
   - pytorch-cpu=2.5.1
   - jupyter=1.1.1
   - requests
   - setuptools
   - torchaudio=2.5.1
   - torchvision=0.20

In [ ]:
# Establish client connection
client = PortalClient("http://localhost:7778", verify=False)
print(client)

NameError: name 'PortalClient' is not defined

# QCArchive single point example with Psi4

QCArchive/QCFractal can be used to run several QM softwares, but the focus of
the present notebook will highlight the popular, open-source Psi4 program. 

As a simple example, we can specify a water dimer geometry and then
create a singlepoint energy entry into the database with the client.

In [ ]:
# Running a single job
mol = Molecule.from_data(
    """
     0 1
     O  -1.551007  -0.114520   0.000000
     H  -1.934259   0.762503   0.000000
     H  -0.599677   0.040712   0.000000
     --
     0 1
     O   1.350625   0.111469   0.000000
     H   1.680398  -0.373741  -0.758561
     H   1.680398  -0.373741   0.758561

     units angstrom
     no_reorient
     symmetry c1
"""
)

client.add_singlepoints(
    [mol],
    "psi4",
    driver="energy",
    method="b3lyp",
    basis="aug-cc-pvdz",
    keywords={"scf_type": "df", "e_convergence": 6, "freeze_core": True},
    tag="local",
)


## Query Results

In [11]:
recs = client.query_records(
    record_id=[1]
)
for rec in recs:
    print(rec.dict)
    print(rec.properties['return_result'])

NameError: name 'client' is not defined

# Dataset Example
While one could use the above logic for all calculations, QCArchive provides
dataset options to make managing data for a specific application substantially
easier. 

Prior to creating the QCArchive datasets, we will first load in data from a 
recently submitted Sherrill work [Levels of SAPT II](https://chemrxiv.org/engage/chemrxiv/article-details/67fe885f6e70d6fb2e033804). To faciliate insertion
into the DB, we will create QCElemental Molecule objects in the pickle file
and extract specific columns to store as variables for usage below. 

In [ ]:
# Creating a QCArchive Dataset...
# Load in a dataset from a recent Sherrill work (Levels of SAPT II)
df_LoS = pd.read_pickle("./combined_df_subset_358.pkl")
print(df_LoS[['Benchmark', 'SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz', 'MP2 IE atz', 'SAPT0 TOTAL ENERGY adz' ]])

# Limit to 100 molecules with maximum of 16 atoms to keep computational cost down
df_LoS['size'] = df_LoS['atomic_numbers'].apply(lambda x: len(x))
df_LoS = df_LoS[df_LoS['size'] <= 16]
df_LoS = df_LoS.sample(100, random_state=42, axis=0).copy()
# df_LoS = df_LoS.sample(50, random_state=42, axis=0).copy()
df_LoS.reset_index(drop=True, inplace=True)
print(df_LoS['size'].describe())

# Create QCElemntal Molecules to generate the dataset
def qcel_mols(row):
    """
    Convert the row to a qcel molecule
    """
    atomic_numbers = [row['atomic_numbers'][row['monAs']], row['atomic_numbers'][row['monBs']]]
    coords = [row['coordinates'][row['monAs']], row['coordinates'][row['monBs']]]
    cm = [
        [row['monA_charge'], row['monA_multiplicity']],
        [row['monB_charge'], row['monB_multiplicity']],
     ]
    return tools.convert_pos_carts_to_mol(atomic_numbers, coords, cm)
df_LoS['qcel_molecule'] = df_LoS.apply(qcel_mols, axis=1)
geoms = df_LoS['qcel_molecule'].tolist()
ref_IEs = df_LoS['Benchmark'].tolist()
sapt0_adz = (df_LoS['SAPT0 TOTAL ENERGY adz'] * h2kcalmol).tolist()

## Singlepoint Dataset

First, to demonstrate how you could use this data for singlepoint energies/properties like the water
dimer example above, we will create a singlepoint dataset and slot in our geometry data. 

In [ ]:
# Create client dataset
ds_name = 'LoS-singlepoint'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name not in client_datasets:
    ds = client.add_dataset("singlepoint", ds_name,
                            f"Dataset to contain {ds_name}")
    print(f"Added {ds_name} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        extras = {
            "name": 'LoS-' + str(idx),
            "idx": idx,
        }
        mol = Molecule.from_data(mol.dict(), extras=extras)
        ent = SinglepointDatasetEntry(name=extras['name'], molecule=mol)
        entry_list.append(ent)
    ds.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds = client.get_dataset("singlepoint", ds_name)
    print(f"Found {ds_name} dataset, using this instead")

print(ds)

In [ ]:
# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=ds.id, delete_records=True)

# Create QCSpecification and submit computations
Now that the dataset has been created with our molecular systems, we will
specify a level of theory (method/basis set) for running our calculations. We
will name our specification "psi4/SAPT0/cc-pvdz" to easily label this
specification's level of theory and program. This name will be used later
during data collection and analysis. Although many QM methods that use a
singlepoint energy would return a total energy, SAPT is a specialized method
that directly computes the interaction energy between our monomers.

In [ ]:
# SAPT0 Example
method, basis = "SAPT0", "cc-pvdz"

# Set the QCSpecification (QM interaction energy in our case)
spec = QCSpecification(
    program="psi4",
    driver="energy",
    method=method,
    basis=basis,
    keywords={
        "scf_type": "df",
    },
)
ds.add_specification(name=f"psi4/{method}/{basis}", specification=spec)

In [ ]:
# Run the computations
ds.submit()
print(f"Submitted {ds_name} dataset")

In [ ]:
# Check the status of the dataset - can repeatedly run this to see the progress
ds.status()

## Manybody Dataset - typical way to compute interaction energies with most QM methods

While SAPT0 returns an interaction energy from a singlepoint energy, most other QM
methods like HF, MP2, CCSD, or DFT would return a total dimer energy ($E_{\rm dimer}$). To
get QCArchive to run the energies of the dimer, monomerA, and monomerB and return the 
energy difference as above for $E_{\rm IE}$, we can use the ManybodyDataset. The setup is
similar to the singlepoint dataset above, but using different objects.

In [ ]:
# Create client dataset
ds_name_mb = 'LoS-manybody'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name_mb not in client_datasets:
    print("Setting up new dataset:", ds_name_mb)
    ds_mb = client.add_dataset("manybody", ds_name_mb,
                            f"Dataset to contain {ds_name_mb}")
    print(f"Added {ds_name_mb} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        ent = ManybodyDatasetEntry(name=f"LoS-IE-{idx}", initial_molecule=mol)
        entry_list.append(ent)
    ds_mb.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds_mb = client.get_dataset("manybody", ds_name_mb)
    print(f"Found {ds_name_mb} dataset, using this instead")

print(ds_mb)

# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=2, delete_records=True)

In [ ]:
ds_mb.status()

# Specify level of theory for Manybody dataset
Similar to above, we can iterate through different levels of theory, create
QCSpecifications, and submit the dataset computations. The major difference here
is that now the "levels" must be specified for the ManybodySpecification. These
"levels" refer to what level of theory and options we want to run our dimer (2)
and monomer (1) computations with. 

In [ ]:
# Set multiple levels of theory - you can add/remove levels as you desire.
# Computational scaling will get quite expensive with better methods and larger
# basis sets

methods = [
    'hf', # 'svwn', # 'pbe', 
]
basis_sets = [
    '6-31g*'
]

for method in methods:
    for basis in basis_sets:
        # Set the QCSpecification (QM interaction energy in our case)
        qc_spec_mb = QCSpecification(
            program="psi4",
            driver="energy",
            method=method,
            basis=basis,
            keywords={
                "d_convergence": 8,
                "scf_type": "df",
            },
        )

        spec_mb = ManybodySpecification(
            program='qcmanybody',
            bsse_correction=['cp'],
            levels={
                1: qc_spec_mb,
                2: qc_spec_mb,
            },
        )
        print("spec_mb", spec_mb)

        ds_mb.add_specification(name=f"psi4/{method}/{basis}", specification=spec_mb)

        # Run the computations
        ds_mb.submit()
        print(f"Submitted {ds_name} dataset")

# Dataset Status
We can re-execute the following cell to see how many computations have completed and how many are running/waiting.

In [ ]:
pp(ds.status())
pp(ds_mb.status())

In [ ]:
ds_mb.detailed_status()

# Data Assembly

NOTE: While you can execute the following blocks before all computations are
complete, it is recommended to wait until all computations are complete to
continue.

QCArchive allows use to write functions that operate on each dataset entry to return exactly what information
we want from every record through the `compile_values` method. The method returns a
pandas dataframe containing our QCSpecification names and our specified columns for each entry 
in our assemble data functions. Here is the Singlepoint data assembly for SAPT calculations. 

In [ ]:
# Singlepoint data assemble
def assemble_singlepoint_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    sapt_energies = np.array([np.nan, np.nan, np.nan, np.nan, np.nan])
    sapt_energies[0] = qcvars['sapt total energy']
    sapt_energies[1] = qcvars['sapt elst energy']
    sapt_energies[2] = qcvars['sapt exch energy']
    sapt_energies[3] = qcvars['sapt ind energy']
    sapt_energies[4] = qcvars['sapt disp energy']
    return (
        record.molecule,
        record.molecule.atomic_numbers,
        record.molecule.geometry * qcel.constants.bohr2angstroms,
        int(record.molecule.molecular_charge),
        record.molecule.molecular_multiplicity,
        sapt_energies,
    )

def assemble_singlepoint_data_value_names():
    return [
        'qcel_molecule',
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity",
        "SAPT Energies",
    ]

df = ds.compile_values(
    value_call=assemble_singlepoint_data,
    value_names=assemble_singlepoint_data_value_names(),
    unpack=True,
)
pp(df.columns.tolist())
df_sapt0 = df['psi4/SAPT0/cc-pvdz']

Below is the assemble data function for our Manybody dataset results:

In [ ]:
def assemble_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    CP_IE = qcvars['results']['cp_corrected_interaction_energy'] * h2kcalmol
    NOCP_IE = qcvars['results'].get('nocp_corrected_interaction_energy', np.nan) * h2kcalmol
    return (
    record.initial_molecule,
    CP_IE,
    NOCP_IE,
    record.initial_molecule.atomic_numbers,
    record.initial_molecule.geometry * qcel.constants.bohr2angstroms,
    int(record.initial_molecule.molecular_charge),
    record.initial_molecule.molecular_multiplicity,
    )

def assemble_data_value_names():
    return [
        'qcel_molecule',
        "CP_IE",
        "NOCP_IE",
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity"
    ]

df_mb = ds_mb.compile_values(
    value_call=assemble_data,
    value_names=assemble_data_value_names(),
    unpack=True,
)

pp(df_mb.columns.tolist())

Now we can pull together this data for analyzing our results. Our group has some
common plotting scripts operating on pandas Dataframes, so we will convert
our computed data into a format that is compatable with these scripts.

In [ ]:
from cdsg_plot import error_statistics

df_sapt0['sapt0 total energes'] = df_sapt0.apply(lambda x: x['SAPT Energies'][0] * h2kcalmol, axis=1)
df_plot = pd.DataFrame(
    {
        "qcel_molecule": df_mb["psi4/pbe/6-31g*"]["qcel_molecule"],
        "HF/6-31G*": df_mb["psi4/hf/6-31g*"]["CP_IE"],
        'SAPT0/cc-pvdz': df_sapt0['sapt0 total energes'].values,
        # "svwn/6-31G*": df_mb["psi4/svwn/6-31g*"]["CP_IE"],
        # "PBE/6-31G*": df_mb["psi4/pbe/6-31g*"]["CP_IE"],
    }
)
# print(df_plot)
id = [int(i[7:]) for i in df_plot.index]
df_plot['id'] = id
df_plot.sort_values(by='id', inplace=True, ascending=True)
df_plot['reference'] = ref_IEs
df_plot['SAPT0/aug-cc-pvdz'] = sapt0_adz
df_plot['HF/6-31G* error'] = (df_plot['HF/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['PBE/6-31G* error'] = (df_plot['PBE/6-31G*'] - df_plot['reference']).astype(float)
# df_plot['svwn/6-31G* error'] = (df_plot['svwn/6-31G*'] - df_plot['reference']).astype(float)
df_plot['SAPT0/cc-pvdz error'] = (df_plot['SAPT0/cc-pvdz'] - df_plot['reference']).astype(float)
df_plot['SAPT0/aug-cc-pvdz error'] = (df_plot['SAPT0/aug-cc-pvdz'] - df_plot['reference']).astype(float)
# print(df_plot[['HF/6-31G*', 'SAPT0/cc-pvdz', 'reference', "SAPT0/aug-cc-pvdz"]])
df_plot = df_plot.dropna(subset=['qcel_molecule', 'HF/6-31G*', 'SAPT0/cc-pvdz', 'SAPT0/aug-cc-pvdz'])
print(df_plot[['HF/6-31G* error', 'SAPT0/cc-pvdz error', "SAPT0/aug-cc-pvdz error"]].describe())

# Plotting the interaction energy errors
The function below will plot violin plots of the error distributions of the
approximate level of theory and our reference. For this dataset, the reference
energies are estimated CCSD(T)/CBS interaction energies provided by the Levels
of SAPT II Supplemental Information.

In [ ]:
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
    },
    output_filename="S22-IE.png",
    figure_size=(6, 6),
    x_label_fontsize=16,
    ylim=(-15, 15),
    rcParams={},
    usetex=False,
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

![S22-IE_violin.png](./S22-IE_violin.png)

# QCMLForge
The previous sections conclude the benchmarking workflow; however, the remaining blocks in this
notebook focus on using this QM data to train models and run inference to compare against QM methods. 

## Interaction energy ML model inference
A specialized atom-pairwise neural network
([AP-Net](https://pubs.rsc.org/en/content/articlehtml/2024/sc/d4sc01029a))
architecture developed in the Sherrill group has been trained on 1.6 million
dimers to predict SAPT0/aug-cc-pV(D+d)Z interaction energies. The model has been
re-implemented and re-trained in PyTorch in QCMLForge for extending in the
future. Below will take a pre-trained AP-Net2 model and predict energies on our
current dimer dataset.  Note that the AP-Net2 architecture relies on first an
AtomModel for predicting multipoles and intramolecular features, prior to more
message passing neural networks for predicting SAPT component energies. As such,
we must load pre-trained AtomModel weights and APNet2Model weights.

In [14]:
import apnet_pt
from apnet_pt.AtomPairwiseModels.apnet2 import APNet2Model
from apnet_pt.AtomModels.ap2_atom_model import AtomModel

atom_model = AtomModel().set_pretrained_model(model_id=0)
ap2 = APNet2Model(atom_model=atom_model.model).set_pretrained_model(model_id=0)
ap2.atom_model = atom_model.model
print(df_plot['qcel_molecule'].tolist())
apnet2_ies_predicted = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16
)

running on the CPU
running on the CPU
self.dataset=None


NameError: name 'df_plot' is not defined

In [ ]:
# AP-Net2 IE
df_plot['APNet2'] = np.sum(apnet2_ies_predicted, axis=1)
df_plot['APNet2 error'] = (df_plot['APNet2'] - df_plot['reference']).astype(float)
#print(df_plot.sort_values(by='APNet2 error', ascending=True)[['APNet2', 'reference']])
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
    },
    output_filename="S22-IE-AP2.png",
    rcParams={},
    usetex=False,
    figure_size=(4, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)
plt.show()

![S22-IE-AP2_violin.png](./S22-IE-AP2_violin.png)

# Transfer Learning
To demonstrate transfer learning, we can use our reference data to then tune our
pre-trained SAPT0 model to predict the CCSD(T)/CBS energies. Then we will
re-plot, evaluating the model performance. Note that in any application, you
should certainly use much more data and careful train/test set selection than
what is done in this notebook. This is purely meant for demonstraion purposes
only.

In [15]:
# Training models on new QM data: Transfer Learning

from apnet_pt import pairwise_datasets

ds2 = pairwise_datasets.apnet2_module_dataset(
    root="data_dir",
    spec_type=None,
    atom_model=atom_model,
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=[np.array([i]) for i in df_plot['reference'].tolist()],
    skip_compile=True,
    force_reprocess=True,
    atomic_batch_size=8,
    prebatched=False,
    in_memory=True,
    batch_size=4,
)
print(ds2)

NameError: name 'df_plot' is not defined

In [ ]:
# Transfer Learning APNet2 model on computed QM data
ap2.train(
    dataset=ds2,
    n_epochs=50,
    transfer_learning=True,
    skip_compile=True,
    model_path="apnet2_transfer_learning.pt",
    split_percent=0.8,
)

In [ ]:
# AP-Net2 IE
apnet2_ies_predicted_transfer = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16,
)
df_plot['APNet2 transfer'] = np.sum(apnet2_ies_predicted_transfer, axis=1)
df_plot['APNet2 transfer error'] = (df_plot['APNet2 transfer'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
    },
    output_filename="S22-IE-AP2-tf.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

![S22-IE_violin-AP2-tf.png](./S22-IE-AP2-tf_violin.png)

## $\Delta$ Models
Another option available in QCMLForge is to train $\Delta {\rm APNet2}$ models 
to predict the difference between one level of theory and another. This
example highlights learning how to go from our computed HF/6-31G* IEs
to our CCSD(T)/CBS energies.

In [ ]:
from apnet_pt.pt_datasets.dapnet_ds import dapnet2_module_dataset_apnetStored

delta_energies = df_plot['HF/6-31G* error'].tolist()

# Only operates in pre-batched mode
ds_dap2 = dapnet2_module_dataset_apnetStored(
    root="data_dir",
    r_cut=5.0,
    r_cut_im=8.0,
    spec_type=None,
    max_size=None,
    force_reprocess=True,
    batch_size=2,
    num_devices=1,
    skip_processed=False,
    skip_compile=True,
    print_level=2,
    in_memory=True,
    m1="HF/6-31G*",
    m2="CCSD(T)/CBS",
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=delta_energies,
)
print(ds_dap2)

In [ ]:
from apnet_pt.AtomPairwiseModels.dapnet2 import dAPNet2Model

dap2 = dAPNet2Model(
    atom_model=AtomModel().set_pretrained_model(model_id=0),
    apnet2_model=APNet2Model().set_pretrained_model(model_id=0).set_return_hidden_states(True),
)
dap2.train(
    ds_dap2,
    n_epochs=50,
    skip_compile=True,
    split_percent=0.6,
)

In [ ]:
dAPNet2_ies_predicted_transfer = dap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=2,
)
df_plot['dAPNet2'] = dAPNet2_ies_predicted_transfer
df_plot['HF/6-31G*-dAPNet2'] = df_plot['HF/6-31G*'] - df_plot['dAPNet2']
print(df_plot[['dAPNet2', 'HF/6-31G*', 'HF/6-31G*-dAPNet2',  'reference']])
df_plot['dAPNet2 error'] = (df_plot['HF/6-31G*-dAPNet2'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        # "svwn/6-31G*": "svwn/6-31G* error",
        # "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
        "dAPNet2 HF/6-31G* to CCSD(T)/CBS": "dAPNet2 error",
    },
    output_filename="S22-IE-AP2-dAP2.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

In [ ]:
# Be careful with this for it can corrupt running status...
# !ps aux | grep qcfractal | awk '{ print $2 }' | xargs kill -9

![S22-IE_violin-AP2-dAP2.png](./S22-IE-AP2-dAP2_violin.png)

# The end...

This notebook has been designed to help demonstrate how QCArchive can be used to
compute QM properties through cybershuttle, and then leverage that data for
analysis or training AP-Net based models through QCMLForge. 

If you have any questions about this notebook or used packages, please feel
free to reach out to me at awallace43@gatech.edu!